<a href="https://colab.research.google.com/github/rohitkayastha200-source/Neural-Network-roulette-feature-probability-/blob/main/IRN%20V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# Roulette Feature Neuron Network
# Full MNIST – NumPy Only
# ===============================

import numpy as np
import urllib.request
import gzip
import os
import random

# -------------------------------
# 1. MNIST Loader (Colab-safe)
# -------------------------------
BASE_URL = "https://ossci-datasets.s3.amazonaws.com/mnist/"

def load_mnist_images(filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve(BASE_URL + filename, filename)
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
    return data.reshape(-1, 784).astype(np.float32) / 255.0

def load_mnist_labels(filename):
    if not os.path.exists(filename):
        urllib.request.urlretrieve(BASE_URL + filename, filename)
    with gzip.open(filename, 'rb') as f:
        labels = np.frombuffer(f.read(), np.uint8, offset=8)
    return labels.astype(int)

print("Loading MNIST...")
X = load_mnist_images("train-images-idx3-ubyte.gz")
y = load_mnist_labels("train-labels-idx1-ubyte.gz")

# Optional low-spec limit
X = X[:20000]
y = y[:20000]

# -------------------------------
# 2. 20 Cheap Feature Extractor
# -------------------------------
def extract_features(image):
    img = image.reshape(28, 28)
    features = []

    # Vertical projections (3)
    features.append(img[:, :9].sum())
    features.append(img[:, 9:18].sum())
    features.append(img[:, 18:].sum())

    # Horizontal projections (3)
    features.append(img[:9, :].sum())
    features.append(img[9:18, :].sum())
    features.append(img[18:, :].sum())

    # Diagonals (2)
    features.append(np.trace(img))
    features.append(np.trace(np.fliplr(img)))

    # Quadrants (4)
    features.append(img[:14, :14].sum())
    features.append(img[:14, 14:].sum())
    features.append(img[14:, :14].sum())
    features.append(img[14:, 14:].sum())

    # Center of mass (2)
    ys, xs = np.indices(img.shape)
    total = img.sum() + 1e-8
    features.append((xs * img).sum() / total)
    features.append((ys * img).sum() / total)

    # Edge density (4)
    features.append(np.abs(np.diff(img, axis=0)).sum())
    features.append(np.abs(np.diff(img, axis=1)).sum())
    features.append(np.abs(np.diff(img[:14], axis=1)).sum())
    features.append(np.abs(np.diff(img[14:], axis=1)).sum())

    # Loop detectors (2)
    features.append(img[10:18, 10:18].sum())
    features.append(img[12:16, 12:16].sum())

    features = np.array(features, dtype=np.float32)
    return features / (np.linalg.norm(features) + 1e-8)

print("Extracting features...")
FEATURES = np.array([extract_features(x) for x in X])

mask = np.isin(y, [0,1,7])
FEATURES = FEATURES[mask]
y = y[mask]


print(f"Using {len(y)} samples (only digits 0,1,7) for fast training")

# -------------------------------
# 3. Roulette Neuron Definition
# -------------------------------
class RouletteNeuron:
    def __init__(self, feature_size=20, num_classes=10):
        self.feature = np.zeros(feature_size, dtype=np.float32)
        self.prob = np.ones(num_classes, dtype=np.float32) / num_classes
        self.occupied = False

# -------------------------------
# 4. Network Init
# -------------------------------
NUM_NEURONS = 300
neurons = [RouletteNeuron() for _ in range(NUM_NEURONS)]

# -------------------------------
# 5. Training (WITH punishment)
# -------------------------------
def train(neurons, feature, label, lr=0.1):
    sims = np.array([
        np.dot(n.feature, feature) if n.occupied else 0.0
        for n in neurons
    ])

    if sims.sum() == 0:
        for n in neurons:
            if not n.occupied:
                n.feature = feature.copy()
                n.prob[:] = 0
                n.prob[label] = 1.0
                n.occupied = True
                return
        return

    probs = sims / sims.sum()
    idx = np.random.choice(len(neurons), p=probs)
    n = neurons[idx]

    predicted = np.argmax(n.prob)

    # 🔥 THE ONE RULE
    if predicted != label:
        n.prob[predicted] *= 0.7

    # Forgetting
    n.prob *= 0.995

    # Reward correct class
    n.prob[label] += lr

    # Normalize
    n.prob /= n.prob.sum()

    # Feature update
    n.feature = 0.9 * n.feature + 0.1 * feature
    n.occupied = True

# -------------------------------
# 6. Prediction
# -------------------------------
def predict(neurons, feature):
    votes = np.zeros(10)
    for n in neurons:
        if n.occupied:
            sim = np.dot(n.feature, feature)
            votes += sim * n.prob
    return np.argmax(votes)

# -------------------------------
# 7. Infinite Roulette Training
# -------------------------------
TARGET_ACC = 0.90
BATCH_EVAL = 300
EVAL_EVERY = 2000
iteration = 0

print("Starting training...")

while True:
    iteration += 1
    i = random.randint(0, len(FEATURES) - 1)
    train(neurons, FEATURES[i], y[i], lr=0.1)

    if iteration % EVAL_EVERY == 0:
        idxs = np.random.choice(len(FEATURES), BATCH_EVAL, replace=False)
        preds = [predict(neurons, FEATURES[i]) for i in idxs]
        acc = np.mean(np.array(preds) == y[idxs])
        print(f"Iteration {iteration} | Sample Accuracy: {acc*100:.2f}%")

        if acc >= TARGET_ACC:
            print("✅ TARGET REACHED")
            break


Loading MNIST...
Extracting features...
Using 6368 samples (only digits 0,1,7) for fast training
Starting training...
Iteration 2000 | Sample Accuracy: 29.67%
Iteration 4000 | Sample Accuracy: 35.33%
Iteration 6000 | Sample Accuracy: 37.00%
Iteration 8000 | Sample Accuracy: 31.00%
Iteration 10000 | Sample Accuracy: 34.33%
Iteration 12000 | Sample Accuracy: 33.67%
Iteration 14000 | Sample Accuracy: 35.33%
Iteration 16000 | Sample Accuracy: 28.00%
Iteration 18000 | Sample Accuracy: 31.67%
Iteration 20000 | Sample Accuracy: 36.33%
Iteration 22000 | Sample Accuracy: 31.67%
Iteration 24000 | Sample Accuracy: 34.00%
Iteration 26000 | Sample Accuracy: 31.00%
Iteration 28000 | Sample Accuracy: 35.67%
Iteration 30000 | Sample Accuracy: 27.00%
Iteration 32000 | Sample Accuracy: 41.33%
Iteration 34000 | Sample Accuracy: 32.67%
Iteration 36000 | Sample Accuracy: 37.67%
Iteration 38000 | Sample Accuracy: 27.67%
Iteration 40000 | Sample Accuracy: 36.00%
Iteration 42000 | Sample Accuracy: 38.67%
Iter